In [ ]:
!git clone https://github.com/NNHieu/INT3405_TGSSalt.git
!pip install -q timm

In [ ]:
!mkdir -p data/train
!unzip -q ../input/tgs-salt-identification-challenge/train.zip -d data/train
!cp ../input/tgssaltdata/folds-2.csv data/folds.csv
!cp ../input/tgs-salt-identification-challenge/depths.csv data/depths.csv

In [ ]:
# import wandb
# wandb.login(key='Key')

In [ ]:
import os
os.chdir('./INT3405_TGSSalt/code')

In [ ]:
from train import parse_args, main
import torch
import gc

In [ ]:
# Do để test nên sẽ chỉ chạy một vài batch, không log và save checkpoint

args = """
--logger_type none
--exp tuning
--version exp_fpa_10
--log_dir ../logs/exp/
--seed 42
--save_checkpoint y

--data_root ../../data
--augment_strategy 6
--add_depth f
--resize_pad f
--num_workers 2
--batch_size 16
--val_fold_idx 1

--model effunet_b4
--loss_func lovasz_hinge_now

--optimizer sgd
--snapshot_size 50
--max_lr 1.5e-2
--min_lr 1e-3
--momentum 0.9
--weight_decay 1e-4
--swa_epoch_start 95
--max_epoch 100
--gpus 1
--progress_bar_refresh_rate 20
--num_sanity_val_steps 2
""".split() + [
    '--limit_train_batches', '10',
    '--limit_val_batches', '10',
]

args = parse_args(args)
dm, model, trainer = main(args)
trainer.fit(model, dm)
# del dm, model, trainer
# gc.collect()
trainer.logger.close()
wandb.finish()

In [ ]:
!rm -r ../../data
!mv -r ../../INT3405_TGSSalt/params ../../params
!rm -r ../../INT3405_TGSSalt/code